# Data Pre Processing

Initial Pre-Processing of data

In [5]:
import pandas as pd
import numpy as np
import re
import pickle

# Load raw data from pickle into a dataframe
df = pd.read_pickle('raw_data.pkl')

# Remove useless columns: anime_url, image_url, scored_by, favorites, rank
columns_to_drop = [
    'anime_url', 'image_url', 'scored_by', 'favorites', 'rank',
    ]
df = df.drop(columns=columns_to_drop, axis=1)

# Remove Special Characters
def clean_text(text):
    if pd.isna(text):
        return ''
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', ' ', text)   # remove punctuation and symbols
    text = re.sub(r'\s+', ' ', text).strip()   # normalize whitespace
    return text


for col in df.select_dtypes(include='object').columns:
    if col == 'synopsis':
      continue
    df[col] = df[col].astype(str).apply(clean_text)


# Change premiered to just year
def extract_year(premiered):
    if pd.isna(premiered):
        return np.nan
    match = re.search(r'(19|20)\d{2}', str(premiered))
    return int(match.group(0)) if match else np.nan

df['premiered'] = df['premiered'].apply(extract_year)

# Change duration to an integer of total minutes
def parse_duration(duration):
    if pd.isna(duration):
        return np.nan
    duration = duration.lower()

    hours = re.search(r'(\d+)\s*hr', duration)
    mins = re.search(r'(\d+)\s*min', duration)

    total_minutes = 0
    if hours:
        total_minutes += int(hours.group(1)) * 60
    if mins:
        total_minutes += int(mins.group(1))

    return total_minutes if total_minutes > 0 else np.nan

df['duration'] = df['duration'].apply(parse_duration)

# Make numbers integers
df['score'] = pd.to_numeric(df['score'], errors='coerce')
df['episodes'] = pd.to_numeric(df['episodes'], errors='coerce')

# Clipping data at the 99th percentile to stop outliers
cols_to_clip = ['members', 'popularity', 'episodes']

for col in cols_to_clip:
    upper = df[col].quantile(0.99)
    lower = df[col].quantile(0.01)
    df[col] = df[col].clip(lower=lower, upper=upper)

# Fill missing data with blanks
df = df.fillna({
    'english_name': '',
    'japanese_names': '',
    'genres': '',
    'themes': '',
    'demographics': '',
    'duration': df['duration'].median(),
    'synopsis': '',
    'episodes': df['episodes'].median(),
    'studios': '',
    'source': '',
    'producers': '',
    'premiered': 0,
    'type': '',
    'rating': '',
})

# Drop useless, almost blank animes
df = df.dropna(subset=['name', 'score'])
df = df[(df['genres'] != '') | (df['synopsis'] != '')]

# Drop duplicate anime entries based on anime_id
df = df.drop_duplicates(subset='anime_id', keep='first').reset_index(drop=True)

# Cleaning up spacing and formatting of some data
# Making commas spaced evenly, forcing all to lowercase
def clean_list_column(text):
    if pd.isna(text): return ''
    return ', '.join([x.strip().lower() for x in str(text).split(',')])

# Applying formatting rules to necessary areas
for col in ['genres', 'themes', 'demographics', 'studios']:
    df[col] = df[col].apply(clean_list_column)

# Saving Cleaned Data
df.to_pickle('cleaned_data.pkl')
df.to_parquet("cleaned_data.parquet", engine='pyarrow', compression='snappy')


UnpicklingError: pickle data was truncated

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [104]:
import pandas as pd
import numpy as np
import re
from rapidfuzz import fuzz
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
from scipy.sparse import hstack

# === 1️⃣ Load cleaned data ===
df = pd.read_parquet("cleaned_data.parquet")

# (Optional) Ensure text columns are strings
for col in ['genres', 'themes', 'demographics', 'synopsis', 'studios', 'type', 'rating']:
    df[col] = df[col].astype(str)

# === 2️⃣ Build TF-IDF vectorizers for different fields ===
vectorizer_genres = TfidfVectorizer(stop_words='english')
vectorizer_themes = TfidfVectorizer(stop_words='english')
vectorizer_synopsis = TfidfVectorizer(stop_words='english', max_features=10000)
vectorizer_studios = TfidfVectorizer(stop_words='english')

# Fit & transform each field
tfidf_genres = vectorizer_genres.fit_transform(df['genres'])
tfidf_themes = vectorizer_themes.fit_transform(df['themes'])
tfidf_synopsis = vectorizer_synopsis.fit_transform(df['synopsis'])
tfidf_studios = vectorizer_studios.fit_transform(df['studios'])

# Normalize to make sure scales are comparable
tfidf_genres = normalize(tfidf_genres)
tfidf_themes = normalize(tfidf_themes)
tfidf_synopsis = normalize(tfidf_synopsis)
tfidf_studios = normalize(tfidf_studios)

# === 3️⃣ Set adjustable weights for each feature group ===
W_GENRES  = 2.0    # importance of genre similarity
W_THEMES  = 1.5    # importance of themes
W_SYNOPSIS = 1.0   # importance of synopsis
W_STUDIOS = 0.5    # importance of studio similarity

# Weighted combination of TF-IDF matrices
combined_matrix = hstack([
    W_GENRES * tfidf_genres,
    W_THEMES * tfidf_themes,
    W_SYNOPSIS * tfidf_synopsis,
    W_STUDIOS * tfidf_studios
])

def extract_core_name(title: str) -> str:
    """
    Extracts a normalized franchise/base title.
    This removes sequel indicators, subtitles, punctuation, and numbers.
    """
    if pd.isna(title):
        return ''
    title = title.lower()

    # Cut at common subtitle separators
    title = re.split(r'[:\-–—\(\)\[\]]', title)[0]

    # Remove sequel indicators
    title = re.sub(r'\b(season|part|special|movie|ova|final|chapter|edition|arc|remake|rebuild|stage|act|sequel|prequel|series)\b', '', title)

    # Remove ordinal indicators like 1st, 2nd, 3rd, etc.
    title = re.sub(r'\b(\d+(st|nd|rd|th))\b', '', title)

    # Remove standalone digits and roman numerals (I, II, III, IV, etc.)
    title = re.sub(r'\b(\d+|i{1,3}|iv|v|vi{0,3}|vii{0,3}|ix|x)\b', '', title)

    # Remove extra words like "hyouketsu no kizuna", "divide by zero"
    # (They are post-subtitle phrases in many MAL entries)
    title = re.sub(r'(no|by|kizuna|link|divide|zero|hyouketsu)', '', title)

    # Remove punctuation, normalize whitespace
    title = re.sub(r'[^a-z0-9\s]', ' ', title)
    title = re.sub(r'\s+', ' ', title).strip()

    return title

df["core_name"] = df["name"].apply(extract_core_name)

# === 4️⃣ Compute cosine similarity ===
cosine_sim = cosine_similarity(combined_matrix, combined_matrix)

print("✅ Combined TF-IDF matrix and cosine similarity computed.")


# === 5️⃣ Recommendation function ==
def recommend_anime_by_id(anime_id, top_n=5):
    """Recommend similar anime given its unique anime_id, excluding sequels/franchise duplicates."""
    if anime_id not in df['anime_id'].values:
        return f"No anime found with id {anime_id}."

    idx = df.index[df['anime_id'] == anime_id][0]
    base_name = df.loc[idx, 'name']
    base_core = df.loc[idx, 'core_name']

    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    recs = []
    for i, _ in sim_scores:
        if i == idx:
            continue

        cand_core = df.loc[i, 'core_name']
        cand_name = df.loc[i, 'name']

        # Skip same or very similar franchises
        if cand_core == base_core:
            continue
        if base_core in cand_core or cand_core in base_core:
            continue
        if fuzz.partial_ratio(base_core, cand_core) > 75:
            continue
        if re.search(r'(season|part|ova|special|movie|final|rebuild)', cand_name, re.I):
            if base_core.split()[0] in cand_core:
                continue

        recs.append(i)
        if len(recs) >= top_n * 3:  # collect a few extra to filter later
            break

    results = df.iloc[recs][['anime_id', 'name', 'core_name', 'score', 'genres', 'rating']]
    results = results.drop_duplicates(subset='core_name', keep='first').head(top_n)

    # ✅ Return only the clean version too
    return results[['anime_id', 'name', 'score']].reset_index(drop=True)

print(recommend_anime_by_id(9253, top_n=5))


✅ Combined TF-IDF matrix and cosine similarity computed.
   anime_id                                               name  score
0     42203  re zero kara hajimeru isekai seikatsu 2nd seas...   8.42
1     49241  vivy fluorite eye s song to make everyone happ...   7.31
2     46095                           vivy fluorite eye s song   8.38
3       385                                          gilgamesh   6.59
4     39584                         human lost ningen shikkaku   5.78


Preprocessing w/ out name bias

In [31]:
import pandas as pd
import numpy as np
import re
import pickle

# Load raw data from pickle into a dataframe
df = pd.read_pickle('raw_data.pkl')

# Remove useless columns: anime_url, image_url, scored_by, favorites, rank, and both names
columns_to_drop = [
    'anime_url', 'image_url', 'scored_by', 'favorites', 'rank', 'english_name', 'japanese_names'
    ]
df = df.drop(columns=columns_to_drop, axis=1)

print(df.shape)

# Remove Special Characters
def clean_text(text):
    if pd.isna(text):
        return ''
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', ' ', text)   # remove punctuation and symbols
    text = re.sub(r'\s+', ' ', text).strip()   # normalize whitespace
    return text


for col in df.select_dtypes(include='object').columns:
    if col == 'synopsis':
      continue
    df[col] = df[col].astype(str).apply(clean_text)


# Change premiered to just year
def extract_year(premiered):
    if pd.isna(premiered):
        return np.nan
    match = re.search(r'(19|20)\d{2}', str(premiered))
    return int(match.group(0)) if match else np.nan

df['premiered'] = df['premiered'].apply(extract_year)

# Change duration to an integer of total minutes
def parse_duration(duration):
    if pd.isna(duration):
        return np.nan
    duration = duration.lower()

    hours = re.search(r'(\d+)\s*hr', duration)
    mins = re.search(r'(\d+)\s*min', duration)

    total_minutes = 0
    if hours:
        total_minutes += int(hours.group(1)) * 60
    if mins:
        total_minutes += int(mins.group(1))

    return total_minutes if total_minutes > 0 else np.nan

df['duration'] = df['duration'].apply(parse_duration)

# Make numbers integers
df['score'] = pd.to_numeric(df['score'], errors='coerce')
df['episodes'] = pd.to_numeric(df['episodes'], errors='coerce')

# Clipping data at the 99th percentile to stop outliers
cols_to_clip = ['members', 'popularity', 'episodes']

for col in cols_to_clip:
    upper = df[col].quantile(0.99)
    lower = df[col].quantile(0.01)
    df[col] = df[col].clip(lower=lower, upper=upper)

# Fill missing data with blanks
df = df.fillna({
    'genres': '',
    'themes': '',
    'demographics': '',
    'duration': df['duration'].median(),
    'synopsis': '',
    'episodes': df['episodes'].median(),
    'studios': '',
    'source': '',
    'producers': '',
    'premiered': 0,
    'type': '',
    'rating': '',
})

# Drop useless, almost blank animes
df = df.dropna(subset=['score'])
df = df[(df['genres'] != '') | (df['synopsis'] != '')]
df = df[(df['type'] == 'tv') | (df['type'] == 'movie') | (df['type'] == 'ona') | (df['type'] == 'ova')]
df = df[~((df["type"] != "movie") & (df["episodes"] < df['episodes'].median()))]
df = df[(df['score'] > 5)]
df = df[~df["genres"].str.contains("hentai", case=False, na=False)]
df = df[~df["genres"].str.contains("erotica", case=False, na=False)]

# Drop duplicate anime entries based on anime_id
df = df.drop_duplicates(subset='anime_id', keep='first').reset_index(drop=True)

# Cleaning up spacing and formatting of some data
# Making commas spaced evenly, forcing all to lowercase
def clean_list_column(text):
    if pd.isna(text): return ''
    return ', '.join([x.strip().lower() for x in str(text).split(',')])

# Applying formatting rules to necessary areas
for col in ['genres', 'themes', 'demographics', 'studios']:
    df[col] = df[col].apply(clean_list_column)

# Saving Cleaned Data
df.to_pickle('cleaned_data.pkl')
df.to_parquet("cleaned_data.parquet", engine='pyarrow', compression='snappy')

(15000, 17)


In [12]:
df = pd.read_csv('raw_data.csv')
df.to_pickle('raw_data.pkl')

In [18]:
filtered_df = df[df['anime_id'] == 21]
print(filtered_df.head)


<bound method NDFrame.head of Empty DataFrame
Columns: [anime_id, score, genres, themes, demographics, synopsis, type, episodes, premiered, producers, studios, source, duration, rating, popularity, members]
Index: []>
